In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt

In [2]:
#Hyper Parameters
EPOCH = 2
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNISIT = False
train_data = torchvision.datasets.MNIST(
    root ='./mnist',
    train =True,
    transform = torchvision.transforms.ToTensor(),
    download = DOWNLOAD_MNISIT
)

In [27]:
train_loader = Data.DataLoader(dataset=train_data,batch_size=BATCH_SIZE,
                              shuffle=True)
test_data = torchvision.datasets.MNIST(root='./mnist',train=False)
print(test_data.test_data.shape)
    

torch.Size([10000, 28, 28])


In [26]:
test_x = torch.unsqueeze(test_data.test_data,dim=1).type(torch.FloatTensor)[:2000]/255
print(test_x.size(0))
test_y = test_data.test_labels[:2000]
print(test_y.shape[0])

2000
2000


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1, #input shape [1,28,28]
                out_channels=16, #output shape [16,28,28]
                kernel_size = 5,
                stride=1,
                padding=2, #if stride=1 padding=(kernelsize-1)/2
            ),#output [16,28,28]
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2) #[16,14,14]
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,5,1,2), #output [32,14,14]
            nn.ReLU(),
            nn.MaxPool2d(2)
        ) #output shape[32,7,7]
        self.out = nn.Linear(32*7*7,10) 
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0),-1)  # 展平多维的卷积图成 (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output
cnn = CNN()
cnn.cuda()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [21]:
optimizer = torch.optim.Adam(cnn.parameters(),lr=LR)
loss_func = nn.CrossEntropyLoss()
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):   # 分配 batch data, normalize x when iterate train_loader
        b_x=x.cuda() #[50,1,28,28]
        b_y=y.cuda() #[50]
        output = cnn(b_x)               # cnn output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()  
        test_x=test_x.cuda() #[2000,1,28,28]
        test_y=test_y.cuda() #[2000]
        if step%50==0:
            test_ouput = cnn(test_x)
            pred_y = torch.max(test_ouput,1)[1].cuda().data.squeeze()
            accuracy = sum(pred_y.cpu().numpy()==test_y.cpu().numpy())/test_y.size(0)
            print('Epoch:',epoch,'step:',step,'| train loss:',loss.data.cpu().numpy(),
                 '| test accuracy:',accuracy
                 )

Epoch: 0 step: 0 | train loss: 0.006175763 | test accuracy: 0.9835
Epoch: 0 step: 50 | train loss: 0.0056405347 | test accuracy: 0.9835
Epoch: 0 step: 100 | train loss: 0.004600246 | test accuracy: 0.9815
Epoch: 0 step: 150 | train loss: 0.0030800218 | test accuracy: 0.984
Epoch: 0 step: 200 | train loss: 0.0122226365 | test accuracy: 0.9875
Epoch: 0 step: 250 | train loss: 0.008396037 | test accuracy: 0.988
Epoch: 0 step: 300 | train loss: 0.054125234 | test accuracy: 0.9815
Epoch: 0 step: 350 | train loss: 0.0018380416 | test accuracy: 0.9865
Epoch: 0 step: 400 | train loss: 0.0013653827 | test accuracy: 0.9855
Epoch: 0 step: 450 | train loss: 0.053323504 | test accuracy: 0.985
Epoch: 0 step: 500 | train loss: 0.011260573 | test accuracy: 0.9885
Epoch: 0 step: 550 | train loss: 0.058980033 | test accuracy: 0.984
Epoch: 0 step: 600 | train loss: 0.0011107755 | test accuracy: 0.987
Epoch: 0 step: 650 | train loss: 0.036471914 | test accuracy: 0.987
Epoch: 0 step: 700 | train loss: 0.02

In [28]:
test_output = cnn(test_x[:10])
pred_y = torch.max(test_output, 1)[1]#.data.squeeze()
print(pred_y.cpu().numpy(), 'prediction number')
print(test_y[:10].cpu().numpy(), 'real number')


NameError: name 'cnn' is not defined

在x y后加.cuda()
但是GPU的tensor不能numpy()
需要转成cpu后在numpy()